In [1]:
from numpy import sin, cos, tan, pi

def tand(theta):
	return tan(theta*pi/180)

def sind(theta):
	return sin(theta*pi/180)

def cosd(theta):
	return cos(theta*pi/180)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python 3.11 from "/Users/yunxiaozang/opt/anaconda3/envs/delta_arm/bin/python"
  * The NumPy version is: "2.4.0"

and make sure that they are the versions you expect.

Please carefully study the information and documentation linked above.
This is unlikely to be a NumPy issue but will be caused by a bad install
or environment on your machine.

Original error was: dlopen(/Users/yunxiaozang/opt/anaconda3/envs/delta_arm/lib/python3.11/site-packages/numpy/_core/_multiarray_umath.cpython-311-darwin.so, 0x0002): tried: '/Users/yunxiaozang/opt/anaconda3/envs/delta_arm/lib/

In [ ]:
from SimpleMath import tand, sind, cosd

import numpy as np 
# import matplotlib.pyplot as plt 
# import time 
import math 
from numpy import pi


class DeltaKinematics: 
	def __init__(self, rod_b = 0.1, rod_ee = 0.2, r_b = 0.074577, r_ee = 0.02495):
		'''
		configs the robot
		rod_B = length of the link connected to the base
		rod_B = length of the link connected to the end-effector
		r_b   = radius of the base			(distance from center to pin joints)
		r_ee  = radius of the end effector 	(distance from center to universal joints)
		'''

		self.rod_b = rod_b
		self.rod_ee = rod_ee
		self.r_b = r_b
		self.r_ee = r_ee
		self.alpha = np.array([0, 120, 240])


	def fk(self, theta):
		# calculate FK, takes theta(deg)

		rod_b = self.rod_b
		rod_ee = self.rod_ee

		theta = np.array(theta) 

		theta1 = theta[0]
		theta2 = theta[1]
		theta3 = theta[2]

		side_ee	= 2/tand(30)*self.r_ee 
		side_b 	= 2/tand(30)*self.r_b

		t = (side_b - side_ee)*tand(30)/2

		y1 = -(t + rod_b*cosd(theta1))
		z1 = -rod_b*sind(theta1)

		y2 = (t + rod_b*cosd(theta2))*sind(30)
		x2 = y2*tand(60)
		z2 = -rod_b*sind(theta2)

		y3 = (t + rod_b*cosd(theta3))*sind(30)
		x3 = -y3*tand(60)
		z3 = -rod_b*sind(theta3)

		dnm = (y2 - y1)*x3 - (y3 - y1)*x2

		w1 = y1**2 + z1**2
		w2 = x2**2 + y2**2 + z2**2
		w3 = x3**2 + y3**2 + z3**2 

		a1 = (z2-z1)*(y3-y1) - (z3-z1)*(y2-y1)
		b1 = -((w2-w1)*(y3-y1) - (w3-w1)*(y2-y1))/2

		a2 = -(z2-z1)*x3 + (z3-z1)*x2
		b2 = ((w2-w1)*x3 - (w3-w1)*x2)/2

		a = a1**2 + a2**2 + dnm**2
		b = 2*(a1*b1 + a2*(b2-y1*dnm) - z1*dnm**2)
		c = (b2 - y1*dnm)**2 + b1**2 + dnm**2*(z1**2 - rod_ee**2)

		d = b**2 - 4*a*c
		if d < 0:
			return -1 

		z0 = -0.5*(b + d**0.5)/a
		x0 = (a1*z0 + b1)/dnm
		y0 = (a2*z0 + b2)/dnm

		return np.array([x0, y0, z0])

	def ik(self, _3d_pose):
		# calculates IK, returns theta(deg)		
		rod_ee = self.rod_ee
		rod_b = self.rod_b
		r_ee = self.r_ee
		r_b = self.r_b 
		alpha = self.alpha 

		F1_pos = ([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
		J1_pos = ([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
		theta = [0, 0, 0]
        
        # Frame Transform
		mat = np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]])
		pos = np.dot(mat,_3d_pose)
		[x0, y0, z0] = pos	
	
		for i in [0, 1, 2]:

			x = x0*cosd(alpha[i]) + y0*sind(alpha[i])
			y = -x0*sind(alpha[i]) + y0*cosd(alpha[i])
			z = z0

			ee_pos = np.array([x, y, z])

			E1_pos = ee_pos + np.array([0, -r_ee, 0])
			E1_prime_pos = np.array([0, E1_pos[1], E1_pos[2]])
			F1_pos[i] = np.array([0, -r_b, 0])

			_x0 = E1_pos[0]
			_y0 = E1_pos[1]
			_z0 = E1_pos[2]
			_yf = F1_pos[i][1]

			c1 = (_x0**2 + _y0**2 + _z0**2 + rod_b**2 - rod_ee**2 - _yf**2)/(2*_z0)
			c2 = (_yf - _y0)/_z0
			c3 = -(c1 + c2*_yf)**2 + (c2**2+ 1)*rod_b**2

			if c3 < 0:
				# print("non existing point")
				return int(-1)

			J1_y = (_yf - c1*c2 - c3**0.5)/(c2**2 + 1)
			J1_z = c1 + c2*J1_y
			F1_y = -r_b

			theta[i] = math.atan(-J1_z/(F1_y - J1_y))*180/pi

		return np.array(theta)


: 